### Data Parsing Setup

In [ ]:
import os
import numpy as np
import pandas as pd

import plotly.graph_objects as go

# import plotly.express as px
# import plotly.subplots as subplots

csv_cols = [
    "dt_us",
    "voltage",
    "heartbeat",
    "wheel_rpm",
    "engine_rpm",
    "target_rpm",
    "velocity_command",
    "real_velocity_command",
    "shadow_count",
    "ignore1",
    "ignore2",
    "iq_measured",
    "flushed",
    "wheel_count",
    "engine_count",
    "iq_setpoint",
    "start_us",
    "stop_us",
    "current",
    "axis_error",
    "motor_error",
    "encoder_error",
]

wheel_diameter = 23
secondary_ratio = 12 / 6 * 45 / 17
wheel_ratio = 12 / 6 * 18 / 57
pitch_angle = 5
encoder_cpr = 8192

In [ ]:
class DescribedDataframe(pd.DataFrame):
    _metadata = ["filename", "description"]

    @property
    def _constructor(self):
        return DescribedDataframe

In [ ]:
def parseCSVFile(path):
    df = pd.read_csv(path, skiprows=1, header=None, names=csv_cols)
    return DescribedDataframe(df)


def parseBinaryFile(path):
    return None


def postProcessDataframe(df):
    df["start_s"] = df["start_us"] / 1e6

    df["secondary_rpm"] = df["wheel_rpm"] * secondary_ratio
    df["wheel_rpm"] = df["wheel_rpm"] * wheel_ratio
    df["wheel_mph"] = (df["wheel_rpm"] * wheel_diameter * np.pi) / (12 * 5280) * 60

    df["actuator_position_mm"] = -df["shadow_count"] / encoder_cpr * pitch_angle
    df["shift_ratio"] = df["secondary_rpm"] / df["engine_rpm"]

    motor_error_idxs = np.where(df["motor_error"] != 0)
    motor_error_bool = np.zeros(len(df))
    motor_error_bool[motor_error_idxs] = 1
    df["motor_error_bool"] = motor_error_bool


def addNormalizedColumns(df):
    for col in df:
        col_norm = f"norm_{col}"
        if col.startswith("norm_") or col_norm in df:
            continue
        col_obj = df[col]
        df[col_norm] = (col_obj - np.min(col_obj)) / (np.max(col_obj) - np.min(col_obj))

### Parse Data

In [ ]:
paths = [
    # "logs/log_2023-04-13_14-03-49.txt",
    # "logs/log_2023-04-13_14-04-12.txt",
    # "logs/log_2023-04-13_20-50-05.txt",
    # "logs/log_2023-04-13_20-51-34.txt",
    # "logs/log_2023-04-13_20-52-02.txt",
    # "logs/log_2023-04-13_21-13-26.txt",
    # "logs/log_2023-04-13_21-13-44.txt",
    # "logs/log_2023-04-13_21-15-18.txt",
    # "logs/log_2023-04-13_21-15-35.txt",
    # "logs/log_2023-04-14_11-01-24.txt",
    # "logs/log_2023-04-14_16-56-05.txt",
    # "logs/log_2023-04-14_16-56-54.txt",
    # "logs/log_2023-04-14_16-59-10.txt",
    # "logs/log_2023-04-14_17-03-28.txt",
    # "logs/log_2023-04-14_17-05-20.txt",
    # "logs/log_2023-04-14_17-06-14.txt",
    # "logs/log_2023-04-14_17-06-58.txt",
    # "logs/log_2023-04-14_17-08-19.txt",
    # "logs/log_2023-04-14_17-10-39.txt",
    # "logs/log_2023-04-14_17-10-58.txt",
    # "logs/log_2023-04-14_17-11-21.txt",
    # "logs/log_2023-04-14_17-14-25.txt",
    # "logs/log_2023-04-14_17-17-17.txt",
    # "logs/log_2023-04-14_17-18-18.txt",
    # "logs/log_2023-04-14_17-18-56.txt",
    # "logs/log_2023-04-14_17-27-30.txt",
    # "logs/log_2023-04-14_17-30-01.txt",
    # "logs/log_2023-04-14_18-00-22.txt",
    # "logs/log_2023-04-14_18-00-50.txt",
    # "logs/log_2023-04-14_18-20-06.txt",
    # "logs/log_2023-04-14_18-22-27.txt",
    # "logs/log_2023-04-14_18-29-41.txt",
    # "logs/log_2023-04-14_18-37-39.txt",
    # "logs/log_2023-04-14_18-40-02.txt",
    # "logs/log_2023-04-14_18-40-21.txt",
    # "logs/log_2023-04-14_18-40-50.txt",
    # "logs/log_2023-04-14_18-57-33.txt",
    # "logs/log_2023-04-14_19-09-36.txt",
    # "logs/log_2023-04-14_19-10-12.txt",
    # "logs/log_2023-04-14_19-10-30.txt",
    "logs/log_2023-04-14_19-46-54_paige-drive-to-garage-spinout-error.txt",
    "logs/log_2023-04-14_20-13-27_paige-brake-check.txt",
]
generate_html = True
offline = False
show_figures = False

In [ ]:
dfs = []

for path in paths:
    dfs.append(parseCSVFile(path))
    postProcessDataframe(dfs[-1])
    dfs[-1].filename = os.path.basename(path)
    addNormalizedColumns(dfs[-1])

### Figure Generators

In [ ]:
def getRPMFigure(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["start_s"], y=df["engine_rpm"], name="Engine RPM"))
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["secondary_rpm"], name="Secondary RPM")
    )
    fig.add_trace(go.Scatter(x=df["start_s"], y=df["target_rpm"], name="Target RPM"))
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="RPM",
        title=f"Engine RPM and Secondary RPM<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [ ]:
def getRPMAndActuatorFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_engine_rpm"], name="Engine RPM")
    )
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_secondary_rpm"], name="Secondary RPM")
    )
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["norm_actuator_position_mm"], name="Actuator Position"
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        title=f"Normalized Engine RPM, Secondary RPM, and Actuator Position<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [ ]:
def getVehicleSpeedFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"],
            y=df["wheel_mph"],
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Vehicle Speed (mph)",
        title=f"Vehicle Speed<br><sup>{df.filename}</sup>",
    )
    return fig

In [ ]:
def getShiftRatioAndAcuatorFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"],
            y=df["norm_actuator_position_mm"],
            name="Actuator Position (mm)",
        )
    )
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_shift_ratio"], name="Shift Ratio")
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        title=f"Normalized Actuator Position and Shift Ratio<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [ ]:
def getVelocityCommandsFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["velocity_command"], name="Unclamped Velocity Command"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["real_velocity_command"], name="Velocity Command"
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Velocity Command",
        title=f"Velocity Command<br><sup>{df.filename}</sup>",
    )
    return fig

In [ ]:
def getShadowCountFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["shadow_count"], name="Shadow Count")
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Shadow Count",
        title=f"Shadow Count<br><sup>{df.filename}</sup>",
    )
    return fig

### Create Invidual Graphs (By Logs)

In [ ]:
def figuresToHTML(figs, filename, offline=False):
    with open(filename, "w") as file:
        file.write("<html><head></head><body>" + "\n")
        for fig in figs:
            font_size_backup = fig.layout.font.size
            fig.update_layout(font={"size": 20})
            inner_html = (
                fig.to_html(include_plotlyjs=(True if offline else "cdn"))
                .split("<body>")[1]
                .split("</body>")[0]
            )
            file.write(inner_html)
            fig.update_layout(font={"size": font_size_backup})
        file.write("</body></html>" + "\n")

In [ ]:
all_figs = []
for df in dfs:
    all_figs.append(
        [
            getRPMFigure(df),
            getRPMAndActuatorFigure(df),
            getVehicleSpeedFigure(df),
            getShiftRatioAndAcuatorFigure(df),
            getVelocityCommandsFigure(df),
            getShadowCountFigure(df),
        ]
    )

In [ ]:
if generate_html:
    for path, figs in zip(paths, all_figs):
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{filename_without_ext}.html"
        figuresToHTML(figs, html_path, offline=offline)

In [ ]:
if show_figures:
    idx = 0
    print(f"Graphing: {paths[idx]}")
    for fig in all_figs[idx]:
        fig.show()

### Create Group Graphs (By Graph Type)

In [ ]:
figure_names_and_funcs = [
    ("rpm", getRPMFigure),
    ("rpm-and-actuator", getRPMAndActuatorFigure),
    ("vehicle-speed", getVehicleSpeedFigure),
    ("shift-ratio-and-acuator", getShiftRatioAndAcuatorFigure),
    ("velocity-command", getVelocityCommandsFigure),
    ("shadow-count", getShadowCountFigure),
]
all_figs = []
for name, func in figure_names_and_funcs:
    figs = []
    for df in dfs:
        figs.append(func(df))
    all_figs.append((name, figs))

In [ ]:
if generate_html:
    for figs in all_figs:
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{figs[0]}.html"
        figuresToHTML(figs[1], html_path, offline=offline)

### Odrive Helpers

In [ ]:
import odrive.enums
from enum import Enum


class ErrorType(Enum):
    AXIS = 0
    MOTOR = 1
    SENSORLESS_ESTIMATOR = 2
    ENCODER = 3
    CONTROLLER = 4


def printErrors(error, error_type, prefix=""):
    error = int(error)
    if error_type == ErrorType.AXIS:
        error_map_prefix = "AXIS_ERROR_"
    elif error_type == ErrorType.MOTOR:
        error_map_prefix = "MOTOR_ERROR_"
    elif error_type == ErrorType.SENSORLESS_ESTIMATOR:
        error_map_prefix = "SENSORLESS_ESTIMATOR_ERROR_"
    elif error_type == ErrorType.ENCODER:
        error_map_prefix = "ENCODER_ERROR_"
    elif error_type == ErrorType.CONTROLLER:
        error_map_prefix = "CONTROLLER_ERROR_"
    error_map = {
        v: k for k, v in odrive.enums.__dict__.items() if k.startswith(error_map_prefix)
    }
    for bit in range(64):
        if error & (1 << bit) != 0:
            print(f"{prefix}{error_map.get(1 << bit)}")

### Error Logging

In [ ]:
error_types = [
    (ErrorType.MOTOR, "motor_error", "Motor Errors"),
    (ErrorType.AXIS, "axis_error", "Axis Errors"),
    (ErrorType.ENCODER, "encoder_error", "Encoder Errors"),
]

In [ ]:
df = dfs[-2]
error_type = error_types[0]

print(f"Errors for {df.filename}:\n")

idxs = np.where(df[error_type[1]] != df[error_type[1]].shift())

for idx in idxs[0]:
    error_time_s = df.iloc[idx]["start_s"]
    error = df.iloc[idx][error_type[1]]
    print(f"{error_type[2]} ({error_time_s:.02f}):")
    if error == 0:
        print("  NONE")
    else:
        printErrors(error, error_type[0], "  ")